In [2]:
# !apt update & apt install -y espeak-ng
# !pip install faster-whisper pydub

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:4 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [47.7 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1245 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Reading package lists... Doneackages 113 kB/1245 kB 9%] [Waiting for headers]0m
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3255 kB]
Building dependency tree... Done] [5 Packages 137 kB/1245 kB 11%] [Waiting for 
Reading state information... Done
espeak-ng is already the newest version (1.50+dfsg-10ubuntu0.1).
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2944 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages 

In [9]:
import os
import re
import gc
import string
import shutil
import traceback
import phonemizer
import pandas as pd
from tqdm import tqdm

from faster_whisper import WhisperModel
from pydub import AudioSegment

In [ ]:
#!rm single-word-voices/train/*
#!rm single-word-voices/val/*

In [ ]:
src = '/storage/services/tts_finetuning/notebooks/ft_datapropio/style_lite/audios_paisa_wav/base2'
model = WhisperModel("base", device="cpu", num_workers=1)

In [5]:
# dir(model)

In [6]:
def remove_punctuation(text):
    words = re.sub('[' + re.escape(''.join(string.punctuation)) + ']', ' ', str(text)).split()
    new = ' '.join(x for x in words)
    return new

my_phonemizer = phonemizer.backend.EspeakBackend(language='es-419', preserve_punctuation=True,  with_stress=True, language_switch='remove-flags')
def get_phoneme(text):
    try:
        return my_phonemizer.phonemize([text])[0]
    except Exception as e:
        print(e)

In [7]:
def delete_tmp():
    tmp_dir = "/tmp"
    for filename in os.listdir(tmp_dir):
        file_path = os.path.join(tmp_dir, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Error al eliminar {file_path}: {e}")

In [ ]:
data = {
    'audio_path': [],
    'phonemes': [],
    'set': [],
    'split': []
}

for set_ in ['train', 'val']:
    with open(os.path.join(src, f'{set_}_f5_centralized.txt'), 'r') as text_file:
        for line in tqdm(text_file):
            try:
                # delete_tmp()
                items = line.split('|')
                if len(items) == 2:
                    data['audio_path'].append(items[0])
                    data['phonemes'].append(items[1])
                    data['set'].append(set_)
                    data['split'].append(False)
                    # if os.path.exists(items[0]):
                    #     try:
                    #         segments, _ = model.transcribe(items[0], word_timestamps=True, language='es')
                    #         audio = AudioSegment.from_wav(items[0])
            
                    #         for i, segment in enumerate(segments):
                    #             for word_info in segment.words:
                    #                 word = word_info.word.strip()
                    #                 label = remove_punctuation(word.lower())
                    #                 if len(label) >= 3:
                    #                     start = word_info.start * 1000  # ms
                    #                     end = word_info.end * 1000      # ms
                    #                     word_audio = audio[start:end]
                    #                     filename = os.path.splitext(os.path.basename(items[0]))[0]
                    #                     output_path = os.path.join('/home/service/StyleTTS2-lite/single-word-voices', set_, f"{filename}_{i:04d}_{label}.wav")
                    #                     word_audio.export(output_path, format="wav")
                    #                     data['audio_path'].append(output_path)
                    #                     data['phonemes'].append(get_phoneme(label))
                    #                     data['set'].append(set_)
                    #                     data['split'].append(True)
                    #     except Exception as e:
                    #         traceback.print_exc()
                    # break
            except Exception as e:
                traceback.print_exc()

for set_ in ['train', 'val']:
    with open(os.path.join(src, f'{set_}.txt'), 'r') as text_file:
        for line in tqdm(text_file):
            try:
                # delete_tmp()
                items = line.split('|')
                if len(items) == 2:
                    data['audio_path'].append(items[0])
                    data['phonemes'].append(items[1])
                    data['set'].append(set_)
                    data['split'].append(False)
                    if os.path.exists(items[0]):
                        try:
                            segments, _ = model.transcribe(items[0], word_timestamps=True, language='es')
                            audio = AudioSegment.from_wav(items[0])
            
                            for i, segment in enumerate(segments):
                                for word_info in segment.words:
                                    word = word_info.word.strip()
                                    label = remove_punctuation(word.lower())
                                    if len(label) >= 3:
                                        start = word_info.start * 1000  # ms
                                        end = word_info.end * 1000      # ms
                                        word_audio = audio[start:end]
                                        filename = os.path.splitext(os.path.basename(items[0]))[0]
                                        output_path = os.path.join('/home/service/StyleTTS2-lite/single-word-voices', set_, f"{filename}_{i:04d}_{label}.wav")
                                        word_audio.export(output_path, format="wav")
                                        data['audio_path'].append(output_path)
                                        data['phonemes'].append(get_phoneme(label))
                                        data['set'].append(set_)
                                        data['split'].append(True)
                        except Exception as e:
                            traceback.print_exc()
                # break
            except Exception as e:
                traceback.print_exc()

In [9]:
df = pd.DataFrame(data).drop_duplicates().sample(frac=1.0)
df

,audio_path,phonemes,set,split
16058,/storage/services/tts_finetuning/notebooks/Sty...,tˈoðo ˈa sˈiðo ɾɾˌeβisˈaðo.\n,train,False
31406,/home/service/StyleTTS2-lite/single-word-voice...,alˈeɣɾa,train,True
18212,/storage/services/tts_finetuning/notebooks/Sty...,¿pwˈeðes βˌeɾifikˈaɾ si ˈaɪ alɣˈun kolˈexjo sˈ...,train,False
11349,/storage/services/tts_finetuning/notebooks/Sty...,fˌiɾaβitˈoβa\n,train,False
510,/storage/services/tts_finetuning/notebooks/Sty...,las mexˈoɾes palˈaβɾas ðel amˈoɾ estˈan ˌentɾe...,train,False
...,...,...,...,...
8490,/storage/services/tts_finetuning/notebooks/Sty...,aβˈaxo\n,train,False
23049,/storage/services/tts_finetuning/notebooks/Sty...,la ɣˈia ðe ˌinstalasjˈon nˈo kˌoɾɾespˈonde kon...,train,False
29346,/home/service/StyleTTS2-lite/single-word-voice...,sˌeɲalˈamos,train,True
20363,/storage/services/tts_finetuning/notebooks/Sty...,los amstˈeɾs kˈomen sˌanaˈoɾjas\n,train,False


In [10]:
df.to_csv('consolidated_data.csv', index=False)

In [11]:
df['set'].value_counts()

set
train    30405
val       2801
Name: count, dtype: int64

In [12]:
df['split'].value_counts()

split
False    26217
True      6989
Name: count, dtype: int64

In [13]:
df.groupby(['set', 'split']).size().to_frame('Count')

Count
set   split       
train False  24829
      True    5576
val   False   1388
      True    1413

In [14]:
for set_ in ['train']:
    cnt = 0
    with open(os.path.join(src, f'{set_}_f5_centralized.txt'), 'r') as text_file:
        for line in tqdm(text_file):
            try:
                items = line.split('|')
                if len(items) == 2:
                    if os.path.exists(items[0]):
                        try:
                            segments, _ = model.transcribe(items[0], word_timestamps=True, language='es')
                            audio = AudioSegment.from_wav(items[0])
            
                            for i, segment in enumerate(segments):
                                for word_info in segment.words:
                                    word = word_info.word.strip()
                                    label = remove_punctuation(word.lower())
                                    if len(label) >= 3:
                                        start = word_info.start * 1000  # ms
                                        end = word_info.end * 1000      # ms
                                        word_audio = audio[start:end]
                                        filename = os.path.splitext(os.path.basename(items[0]))[0]
                                        output_path = os.path.join('/home/service/StyleTTS2-lite/single-word-voices', set_, f"{filename}_{i:04d}_{label}.wav")
                                        word_audio.export(output_path, format="wav")
                                        data['audio_path'].append(output_path)
                                        data['phonemes'].append(get_phoneme(label))
                                        data['set'].append(set_)
                                        data['split'].append(True)
                                        cnt += 1
                        except Exception as e:
                            pass
            except Exception as e:
                pass
            if cnt >= 10000:
                break

870it [13:11,  1.10it/s]


Iterations:

- [X] 1373it [32:55,  1.44s/it]

In [15]:
df_extended = pd.DataFrame(data).drop_duplicates().sample(frac=1.0)
df_extended

,audio_path,phonemes,set,split
26195,/home/service/StyleTTS2-lite/single-word-voice...,kˈon,train,True
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False
37252,/home/service/StyleTTS2-lite/single-word-voice...,dˌesastɾˈenos,train,True
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False
...,...,...,...,...
1769,/storage/services/tts_finetuning/notebooks/Sty...,nˈo nˌesesˈito asˈeɾ ðjˈeta.\n,train,False
1897,/storage/services/tts_finetuning/notebooks/Sty...,templˈansa i tɾaβˈaxo sˈon los ðˈos mexˈoɾes k...,train,False
6817,/storage/services/tts_finetuning/notebooks/Sty...,kaɾɾˈeɾa tɾˈese nˈo. otʃˌɛntaikwˈatɾo βˌeɪntin...,train,False
3768,/storage/services/tts_finetuning/notebooks/Sty...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...,train,False


In [16]:
df_extended.to_csv('consolidated_data_extended.csv', index=False)

In [17]:
df_extended.groupby(['set', 'split']).size().to_frame('Count')

Count
set   split       
train False  24829
      True   14970
val   False   1388
      True    1413

In [18]:
df_extended['keep'] = df_extended['audio_path'].apply(lambda path_: os.path.exists(path_))
df_extended

,audio_path,phonemes,set,split,keep
26195,/home/service/StyleTTS2-lite/single-word-voice...,kˈon,train,True,True
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False,True
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False,True
37252,/home/service/StyleTTS2-lite/single-word-voice...,dˌesastɾˈenos,train,True,True
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False,True
...,...,...,...,...,...
1769,/storage/services/tts_finetuning/notebooks/Sty...,nˈo nˌesesˈito asˈeɾ ðjˈeta.\n,train,False,True
1897,/storage/services/tts_finetuning/notebooks/Sty...,templˈansa i tɾaβˈaxo sˈon los ðˈos mexˈoɾes k...,train,False,True
6817,/storage/services/tts_finetuning/notebooks/Sty...,kaɾɾˈeɾa tɾˈese nˈo. otʃˌɛntaikwˈatɾo βˌeɪntin...,train,False,True
3768,/storage/services/tts_finetuning/notebooks/Sty...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...,train,False,True


In [19]:
df_extended['keep'].value_counts()

keep
True     42598
False        2
Name: count, dtype: int64

In [20]:
df_filter = df_extended[df_extended['keep']]
df_filter

,audio_path,phonemes,set,split,keep
26195,/home/service/StyleTTS2-lite/single-word-voice...,kˈon,train,True,True
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False,True
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False,True
37252,/home/service/StyleTTS2-lite/single-word-voice...,dˌesastɾˈenos,train,True,True
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False,True
...,...,...,...,...,...
1769,/storage/services/tts_finetuning/notebooks/Sty...,nˈo nˌesesˈito asˈeɾ ðjˈeta.\n,train,False,True
1897,/storage/services/tts_finetuning/notebooks/Sty...,templˈansa i tɾaβˈaxo sˈon los ðˈos mexˈoɾes k...,train,False,True
6817,/storage/services/tts_finetuning/notebooks/Sty...,kaɾɾˈeɾa tɾˈese nˈo. otʃˌɛntaikwˈatɾo βˌeɪntin...,train,False,True
3768,/storage/services/tts_finetuning/notebooks/Sty...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...,train,False,True


In [21]:
df_filter.groupby(['set', 'split']).size().to_frame('Count')

Count
set   split       
train False  24829
      True   14969
val   False   1388
      True    1412

In [22]:
for set_ in ['train', 'val']:
    df_tmp = df_filter[df_filter['set'] == set_].copy()
    with open(f'Data_Speech/{set_}.txt', 'w') as text_file:
        for idx, row in df_tmp.iterrows():
            n_line = f"{row['audio_path']}|{row['phonemes'].strip()}"
            text_file.write(f'{n_line}\n')

In [23]:
!pip list

Package                   Version
------------------------- --------------
absl-py                   2.2.2
accelerate                1.6.0
anaconda-anon-usage       0.4.4
annotated-types           0.7.0
anyio                     4.9.0
archspec                  0.2.3
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.0.5
astunparse                1.6.3
async-lru                 2.0.5
attrs                     23.1.0
audioread                 3.0.1
av                        14.3.0
babel                     2.17.0
beautifulsoup4            4.12.3
bleach                    6.2.0
boltons                   23.0.0
Brotli                    1.0.9
certifi                   2024.7.4
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        2.0.4
click                     8.1.7
colorama                  0.4.6
coloredlogs               15.0.1
comm                      0.2.2
conda            

In [24]:
def analyze_audio(audio_path):
    segments, _ = model.transcribe(audio_path, language='es')
    transcription = ''
    for segment in segments:
        transcription += segment.text + " "
    return pd.Series([transcription, get_phoneme(transcription)])

In [25]:
df_org = df_filter[~df_filter['split']].copy()
df_org

,audio_path,phonemes,set,split,keep
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False,True
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False,True
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False,True
16122,/storage/services/tts_finetuning/notebooks/Sty...,dˈatos ˌapɾoβˈaðos.\n,train,False,True
21189,/storage/services/tts_finetuning/notebooks/Sty...,si estˈas ˌenoxˈaðo ˈes mexˈoɾ ˌespeɾˈaɾ a ke ...,train,False,True
...,...,...,...,...,...
1769,/storage/services/tts_finetuning/notebooks/Sty...,nˈo nˌesesˈito asˈeɾ ðjˈeta.\n,train,False,True
1897,/storage/services/tts_finetuning/notebooks/Sty...,templˈansa i tɾaβˈaxo sˈon los ðˈos mexˈoɾes k...,train,False,True
6817,/storage/services/tts_finetuning/notebooks/Sty...,kaɾɾˈeɾa tɾˈese nˈo. otʃˌɛntaikwˈatɾo βˌeɪntin...,train,False,True
3768,/storage/services/tts_finetuning/notebooks/Sty...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...,train,False,True


In [26]:
tqdm.pandas()

In [27]:
df_org.head()['audio_path'].progress_apply(lambda audio_path: analyze_audio(audio_path))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.19it/s]


,0,1
29091,Julio Reía y Trorava mientras bailaban en el ...,xˈuljo ɾɾeˈia i tɾoɾˈaβa mjˌentɾas βaɪlˈaβan e...
379,"Ser original es, en cierto modo, estar ponien...","sˈer ˌoɾixinˈal ˈes, en sjˈeɾto mˈoðo, estˈaɾ ..."
585,"No importa cómo te sientas hoy, levantate, pí...","nˈo impˈoɾta kˈomo te sjˈɛntas ˈoɪ, lˌeβantˈat..."
16122,datos aprobados.,dˈatos ˌapɾoβˈaðos.
21189,"Si estás enujado, es mejor esperar a que te c...","si estˈas ˌenuxˈaðo, ˈes mexˈoɾ ˌespeɾˈaɾ a ke..."


In [28]:
df_org.head()

,audio_path,phonemes,set,split,keep
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False,True
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False,True
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False,True
16122,/storage/services/tts_finetuning/notebooks/Sty...,dˈatos ˌapɾoβˈaðos.\n,train,False,True
21189,/storage/services/tts_finetuning/notebooks/Sty...,si estˈas ˌenoxˈaðo ˈes mexˈoɾ ˌespeɾˈaɾ a ke ...,train,False,True


In [ ]:
df_org[['transcription', 'n_phonemes']] = df_org['audio_path'].progress_apply(lambda audio_path: analyze_audio(audio_path))
df_org

 16%|███████████████▉                                                                                 | 4321/26217 [1:02:26<5:24:03,  1.13it/s]

list index out of range


 17%|████████████████                                                                                 | 4352/26217 [1:02:52<4:59:04,  1.22it/s]

In [ ]:
df_org.to_csv('generated_transcription_phonemes.csv', index=False)

In [31]:
df_org

,audio_path,phonemes,set,split,keep,transcription,n_phonemes
29091,/storage/services/tts_finetuning/notebooks/Sty...,ʝˈuli ɾɾeˈia i ʝoɾˈaβa mjˌentɾas βaɪlˈaβa en l...,train,False,True,Julio Reía y Trorava mientras bailaban en el ...,xˈuljo ɾɾeˈia i tɾoɾˈaβa mjˌentɾas βaɪlˈaβan e...
379,/storage/services/tts_finetuning/notebooks/Sty...,sˈer ˌoɾixinˈal ˈes en sjˈeɾto mˈoðo estˈaɾ po...,train,False,True,"Ser original es, en cierto modo, estar ponien...","sˈer ˌoɾixinˈal ˈes, en sjˈeɾto mˈoðo, estˈaɾ ..."
585,/storage/services/tts_finetuning/notebooks/Sty...,"nˈo impˈoɾta kˌomo te sjˈɛntas ˈoɪ, leβˈantate...",train,False,True,"No importa cómo te sientas hoy, levantate, pí...","nˈo impˈoɾta kˈomo te sjˈɛntas ˈoɪ, lˌeβantˈat..."
16122,/storage/services/tts_finetuning/notebooks/Sty...,dˈatos ˌapɾoβˈaðos.\n,train,False,True,datos aprobados.,dˈatos ˌapɾoβˈaðos.
21189,/storage/services/tts_finetuning/notebooks/Sty...,si estˈas ˌenoxˈaðo ˈes mexˈoɾ ˌespeɾˈaɾ a ke ...,train,False,True,"Si estás enujado, es mejor esperar a que te c...","si estˈas ˌenuxˈaðo, ˈes mexˈoɾ ˌespeɾˈaɾ a ke..."
...,...,...,...,...,...,...,...
1769,/storage/services/tts_finetuning/notebooks/Sty...,nˈo nˌesesˈito asˈeɾ ðjˈeta.\n,train,False,True,No necesito hacer dieta.,nˈo nˌesesˈito asˈeɾ ðjˈeta.
1897,/storage/services/tts_finetuning/notebooks/Sty...,templˈansa i tɾaβˈaxo sˈon los ðˈos mexˈoɾes k...,train,False,True,Templanse trabajo son los dos mejores custoie...,templˈanse tɾaβˈaxo sˈon los ðˈos mexˈoɾes kus...
6817,/storage/services/tts_finetuning/notebooks/Sty...,kaɾɾˈeɾa tɾˈese nˈo. otʃˌɛntaikwˈatɾo βˌeɪntin...,train,False,True,"Carrera 13, número 84.29","kaɾɾˈeɾa tɾˈese, nˈumeɾo otʃˌɛntaikwˈatɾo.bˌeɪ..."
3768,/storage/services/tts_finetuning/notebooks/Sty...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...,train,False,True,El suelo mismo debe ser objeto de observación...,el swˈelo mˈismo ðˈeβe sˈer oβxˈeto ðe ˌoβseɾβ...


In [51]:
df_org.loc[15269]['audio_path']

'/storage/services/tts_finetuning/notebooks/StyleTTS2_Finetune/StyleTTS2/Data/wavs24kHz/Juan_2696.wav'

In [53]:
!pwd

/home/service/StyleTTS2-lite


In [32]:
!nvidia-smi

Mon May 12 10:37:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.90.12              Driver Version: 550.90.12      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H100 80GB HBM3          On  |   00000000:44:00.0 Off |                    0 |
| N/A   33C    P0            116W /  700W |   68887MiB /  81559MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [34]:
df_org[df_org['phonemes'].apply(lambda ph: '?' in ph)]

,audio_path,phonemes,set,split,keep,transcription,n_phonemes
20839,/storage/services/tts_finetuning/notebooks/Sty...,¿i ðe kˈe tamˈaɲo kjˈeɾes la βˈolsa?\n,train,False,True,Y de que tamaño quieres la bolsa.,i ðe ke tamˈaɲo kjˈeɾes la βˈolsa.
24903,/storage/services/tts_finetuning/notebooks/Sty...,¿ˈa sˈiðo ˈun dˈia βwˈeno pˌaɾa ustˈed?\n,val,False,True,así un día bueno para usted,asˈi ˈun dˈia βwˈeno pˌaɾa ustˈed
13575,/storage/services/tts_finetuning/notebooks/Sty...,¿se enkwˈɛntɾa en ˈum bwˈen estˈaðo?\n,train,False,True,se encuentra en un buen estado,se enkwˈɛntɾa en ˈum bwˈen estˈaðo
14153,/storage/services/tts_finetuning/notebooks/Sty...,¿desˈea ɾɾˌesiβˈiɾ ˌoɾjentasjˈon mˈeðika tˌele...,train,False,True,desear recibir orientación médica telefónica,dˌeseˈaɾ ɾɾˌesiβˈiɾ ˌoɾjentasjˈon mˈeðika tˌel...
21256,/storage/services/tts_finetuning/notebooks/Sty...,¿nˌesesˈitas βendˈeɾ alɣˈun ˈotɾo moðˈelo ðe ˈ...,train,False,True,Necesitas vender algún otro modelo de auto?,nˌesesˈitas βendˈeɾ alɣˈun ˈotɾo moðˈelo ðe ˈa...
...,...,...,...,...,...,...,...
21602,/storage/services/tts_finetuning/notebooks/Sty...,¿los ˌalakɾˈanes tendɾˈan ˈun sˌiɡnifikˈaðo si...,train,False,True,Los alacranes tendrán un significado simbólico.,los ˌalakɾˈanes tendɾˈan ˈun sˌiɡnifikˈaðo sim...
16352,/storage/services/tts_finetuning/notebooks/Sty...,¿desˈea ɾɾˌesiβˈiɾ ˌimfoɾmasjˈom poɾ koɾɾˈeo ˌ...,train,False,True,desea recibir información por correo electrón...,desˈea ɾɾˌesiβˈiɾ ˌimfoɾmasjˈom poɾ koɾɾˈeo ˌe...
13907,/storage/services/tts_finetuning/notebooks/Sty...,¿desˈea kˌonosˈeɾ las pˌenalˌisasjˈones poɾ pˈ...,train,False,True,desea conocer las penalizaciones por pagos ta...,desˈea kˌonosˈeɾ las pˌenalˌisasjˈones poɾ pˈa...
16462,/storage/services/tts_finetuning/notebooks/Sty...,¿ˈa notˈaðo alɣˈun eɾɾˈoɾ en su fˌaktuɾasjˈon?\n,train,False,True,ha notado algún error en su facturación,ˈa notˈaðo alɣˈun eɾɾˈoɾ en su fˌaktuɾasjˈon


In [35]:
!pwd

/home/service/StyleTTS2-lite
